In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import missingno as msno
import nltk
import string
import re
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

df = pd.read_csv('transformed_data.csv')
clean_text_df = pd.read_csv('clean_text.csv')
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

#df['body_len'] = df['text'].apply(lambda x: len(x) - x.count(" "))
#df['punct%'] = df['text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\InnovusDrac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\InnovusDrac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df.head()

,title,text,value,try,count_punct,try2,count_ws,body_len,twitter
0,‘Verbal Diarrhea’: Trump Gives Most Astonishi...,It s no secret that Donald Trump talks to his ...,False,"[',', '.', ',', ',', '.', '.', '.', ',', ',', ...",1.613216,0.892244,2.928678,1.303728,4
1,China calls for restraint after a U.S. B-1B bo...,BEIJING (Reuters) - China said on Wednesday it...,True,"['(', ')', '-', ',', '.', '.', '-', '-', '.', ...",1.291550,0.857009,1.912931,1.204411,0
2,U.S. Supreme Court takes up major transgender ...,WASHINGTON (Reuters) - The U.S. Supreme Court ...,True,"['(', ')', '-', '.', '.', '-', '.', '.', '.', ...",1.656672,0.896600,3.058773,1.321101,0
3,Cameroon Anglophone regions to shut Nigeria bo...,YAOUNDE (Reuters) - The government of one of C...,True,"['(', ')', '-', '-', '.', '.', ',', '.', ',', ...",1.514820,0.882098,2.701832,1.290223,0
4,Electors Will NOT Be Given Intelligence Brief...,Members of the Electoral College will not be r...,False,"['.', ',', '.', ',', ',', ',', ',', '.', ',', ...",1.479668,0.878383,2.716675,1.290816,0


In [3]:
del df['try']
df['punct%']=df['try2']
del df['try2']
df.head()

,title,text,value,count_punct,count_ws,body_len,twitter,punct%
0,‘Verbal Diarrhea’: Trump Gives Most Astonishi...,It s no secret that Donald Trump talks to his ...,False,1.613216,2.928678,1.303728,4,0.892244
1,China calls for restraint after a U.S. B-1B bo...,BEIJING (Reuters) - China said on Wednesday it...,True,1.291550,1.912931,1.204411,0,0.857009
2,U.S. Supreme Court takes up major transgender ...,WASHINGTON (Reuters) - The U.S. Supreme Court ...,True,1.656672,3.058773,1.321101,0,0.896600
3,Cameroon Anglophone regions to shut Nigeria bo...,YAOUNDE (Reuters) - The government of one of C...,True,1.514820,2.701832,1.290223,0,0.882098
4,Electors Will NOT Be Given Intelligence Brief...,Members of the Electoral College will not be r...,False,1.479668,2.716675,1.290816,0,0.878383


In [4]:
#due to some majore issues with size, I will be reducing the df size
#after trial and error, 25% was found to be operable, 50%+ crashed the computer even with extra steps
def dovide(x):
    n=25
    m=75
    fir =x.tail(int(len(x)*(m/100)))
    x = x.head(int(len(x)*(n/100)))
    return x,fir

In [5]:
df,df2= dovide(df)
df.shape

(11030, 8)

In [6]:
clean_text_df.head()

,title,text,value
0,"['u', 'say', 'deeply', 'concerned', 'kenya', '...","['washington', 'reuters', '', 'u', 'government...",True
1,"['watch', 'sean', 'hannity', 'fan', 'make', 'h...","['keurig', 'decided', 'side', 'soros', 'sean',...",False
2,"['syrian', 'army', 'battle', 'islamic', 'state...","['beirut', 'reuters', '', 'syrian', 'army', 'a...",True
3,"['white', 'house', 'say', 'government', 'shutd...","['washington', 'reuters', '', 'u', 'congress',...",True
4,"['', 'bill', 'maher', 'scorch', 'republican', ...","['since', 'rise', 'presumptive', 'republican',...",False


In [7]:
sum(df['twitter'])

6910

In [8]:
from sklearn.model_selection import train_test_split

# splitting and testing the body text

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df[['text', 'body_len','twitter', 'punct%']], df['value'], test_size=0.3)

In [10]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])

tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%','twitter']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%','twitter']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)


X_train_vect.head()

,body_len,punct%,twitter,0,1,2,3,4,5,6,...,69049,69050,69051,69052,69053,69054,69055,69056,69057,69058
0,1.271957,0.877802,0,0.021379,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.256153,0.879781,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.244576,0.866186,0,0.028670,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.293896,0.888735,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.223140,0.857018,0,0.046618,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X_train_vect.shape

(7721, 69062)

# Model Eval

In [12]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [13]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=False, average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 124.045 / Predict time: 17.406 ---- Precision: 0.987 / Recall: 0.983 / Accuracy: 0.985


In [14]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label=False, average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 2569.722 / Predict time: 8.737 ---- Precision: 0.995 / Recall: 0.993 / Accuracy: 0.994


In [15]:
#So we see the values are close, but the gradient is a better option, unles the time is an issue